<a href="https://colab.research.google.com/github/wf-yamaday/machine-learning/blob/master/%E8%87%AA%E7%84%B6%E8%A8%80%E8%AA%9E%E5%87%A6%E7%90%86/seq2seq2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 自然言語処理

自然言語処理にチャレンジしてみようということで日本語のChatBotを作成することをゴールに始めて見る．

まずは訓練データとなる，**発話文とその返答文**を収集する．


# 名大会話コーパス

日本語母語話者同士の雑談を文字化したコーパスのこと．[ここ](https://mmsrv.ninjal.ac.jp/nucc/)からダウンロードできる．

名大会話コーパスには以下のような特徴がある．

- テキストファイルの最初には@で始まる部分があり，ヘッダーである．ヘッダーには，収録されている会話データの収集日や参加者の特性が記されている．また会話データの終了部分には，@ENDがフッターとして存在している．
- `%:com` から始まる行に補足情報が記されている．
- 各発話の初めには発話者のコードが記されている．発話者のコードは「Mxxx」，「Fxxx」(xは半角数字)で表されており，Mが男性，Fが女性である．
- 聞き取れない部分には＊＊＊で表されている．
- 相づちと思われる発話は()付けされている．
- 一定の長さ以上の沈黙は＜間＞，笑いは＜笑い＞と表されている．
- 発話データ内には，発言者名(「Mxxx」や「Fxxx」)が現れることがある．

これらの特徴を踏まえテキストデータを整形する．




まず使用するライブラリのインポートを行う．

In [0]:
import numpy as np
import csv
import glob
import re

次にgoogle colaboratoryにテキストファイルを読み込む．

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ここまでで，テキストデータを読み込み準備が整った．
次に，データの整形を行う関数を定義する．

データの整形の関数で行うことは，

1. ファイルを1行単位の配列に変換する
1. 先頭が`%com:`の行を排除
1. @から始まらない行に対し，以下の操作を行う．
    - 不明文字`***`を`UNK`に置換する．
    - 発話中に含まれる発言者名を `UNK`に置換する．
    - 

In [0]:
def format_data(fname, data2) :
  f = open(fname, 'r')
  df1 = csv.reader(f)
  data1 = [ v for v in df1]

  print(len(data1))
  #ファイル読み込み
  text = ''
  for i in range(0,len(data1)):
      if len(data1[i]) == 0:
          print('null')
          continue

      s = data1[i][0]
      if s[0:5] == "％ｃｏｍ：" :
          continue
      if s[0]  != '＠' :
          #不明文字をUNKに置き換え
          s = s.replace('＊＊＊','UNK')
          #会話文セパレータ
          if s[0] == 'F' or s[0] == 'M':
              s = 'SSSS'+s[5:]
          if s[0:2] == 'Ｘ：':
              s = 'SSSS'+s[2:]

          s = re.sub('F[0-9]{3}',"UNK",s)
          s = re.sub('M[0-9]{3}',"UNK",s)
          s = s.replace("＊","")
      else :
          continue

      while s.find("（") != -1 :
          start_1 = s.find("（")
          if s.find("）") != -1 :
              end_1 = s.find("）")
              if start_1 >= end_1 :
                  s = s.replace(s[end_1],"")
              else :
                  s = s.replace(s[start_1:end_1+1],"")
              if len(s) == 0 :
                  continue
          else :
              s=s[0:start_1]

      while s.find("［") != -1 :
          start_2 = s.find("［")
          if s.find("］") != -1 :
              end_2=s.find("］")
              s=s.replace(s[start_2:end_2+1],"")
          else :
              s=s[0:start_2]    

      while s.find("＜") != -1 :
          start_3 = s.find("＜")
          if s.find("＞") != -1 :
              end_3 = s.find("＞")
              s = s.replace(s[start_3:end_3+1],"")
          else :
              s = s[0:start_3] 

      while s.find("【") != -1 :
          start_4 = s.find("【")
          if s.find("】") != -1 :
              end_4 = s.find("】")
              s = s.replace(s[start_4:end_4+1],"")
          else :
              s = s[0:start_4] 

      #いろいろ削除したあとに文字が残っていたら出力文字列に追加
      if s != "\n" and s != "SSSS" :
          text += s
  #セパレータごとにファイル書き込み
  text =text[4:]
  while text.find("SSSS") != -1 :
      end_s = text.find("SSSS")
      t = text[0:end_s]
      #長い会話文を分割
      if end_s > 100 :
          while len(t) > 100 :
              if t.find("。") != -1 :
                  n_period = t.find("。")
                  data2.append("SSSS"+t[0:n_period+1])
                  t = t[n_period+1:]
              else :
                  break
      data2.append("SSSS"+t)
      text = text[end_s+4:]
  f.close()
  return

これを発話が記録されてファイルごとに実行していく．
今回は，自分のGoogle Driveのルートディレクトリ配下に`nucc`ディレクトリを配備している．

In [0]:
file_list = glob.glob('drive/My Drive/nucc/*')
print(len(file_list))

129


In [0]:
print(file_list[1])
data_test = []
format_data(file_list[1], data_test)
f = open('drive/My Drive/nucc2/corpus_test.txt','w')
f.write(str(data_test + "\n")

f.close()
print(len(data_test))

SyntaxError: ignored

In [0]:
data2=[]
for j in range(0,len(file_list)) :
    print(file_list[j])
    format_data(file_list[j],data2)


drive/My Drive/nucc/data033.txt
693
drive/My Drive/nucc/data032.txt
462
drive/My Drive/nucc/data035.txt
1445
null
drive/My Drive/nucc/data036.txt
1250
drive/My Drive/nucc/data037.txt
1843
null
drive/My Drive/nucc/data034.txt
517
drive/My Drive/nucc/data031.txt
1122
drive/My Drive/nucc/data019.txt
688
drive/My Drive/nucc/data020.txt
869
drive/My Drive/nucc/data015.txt
829
drive/My Drive/nucc/data016.txt
953
drive/My Drive/nucc/data018.txt
533
drive/My Drive/nucc/data017.txt
820
drive/My Drive/nucc/data014.txt
1277
drive/My Drive/nucc/data025.txt
1650
drive/My Drive/nucc/data027.txt
469
drive/My Drive/nucc/data023.txt
1096
drive/My Drive/nucc/data026.txt
1063
drive/My Drive/nucc/data030.txt
572
drive/My Drive/nucc/data022.txt
1505
drive/My Drive/nucc/data024.txt
2001
drive/My Drive/nucc/data029.txt
678
drive/My Drive/nucc/data028.txt
894
drive/My Drive/nucc/data021.txt
1013
drive/My Drive/nucc/data008.txt
1561
drive/My Drive/nucc/data006.txt
1358
drive/My Drive/nucc/data007.txt
805
drive

In [0]:
f = open('drive/My Drive/nucc2/corpus.txt','w')
for i in range(0,len(data2)):
    f.write(str(data2[i])+"\n")
f.close()
print(len(data2))

83049


In [0]:
!cat drive/My Drive/nucc2/corpus.txt

cat: drive/My: No such file or directory
cat: Drive/nucc2/corpus.txt: No such file or directory
